In [7]:
import os
import random
import sys
import torch
import math
import pickle
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.data import DataLoader, Dataset ,ConcatDataset
from Data.MyDataSet import MultiMaskTimeSeriesDataset
sys.path.append(r'D:\WorkPath\Models\ImputeFormer\model')

from ImputeFormer_main import ImputeFormer 

In [8]:
class CosineSchedulerWithRestarts(LambdaLR):

    def __init__(self, optimizer: Optimizer,
                 num_warmup_steps: int,
                 num_training_steps: int,
                 min_factor: float = 0.1,
                 linear_decay: float = 0.67,
                 num_cycles: int = 1,
                 last_epoch: int = -1):
        """From https://github.com/huggingface/transformers/blob/v4.18.0/src/transformers/optimization.py#L138

        Create a schedule with a learning rate that decreases following the values
        of the cosine function between the initial lr set in the optimizer to 0,
        with several hard restarts, after a warmup period during which it increases
        linearly between 0 and the initial lr set in the optimizer.

        Args:
            optimizer ([`~torch.optim.Optimizer`]):
                The optimizer for which to schedule the learning rate.
            num_warmup_steps (`int`):
                The number of steps for the warmup phase.
            num_training_steps (`int`):
                The total number of training steps.
            num_cycles (`int`, *optional*, defaults to 1):
                The number of hard restarts to use.
            last_epoch (`int`, *optional*, defaults to -1):
                The index of the last epoch when resuming training.
        Return:
            `torch.optim.lr_scheduler.LambdaLR` with the appropriate schedule.
        """

        def lr_lambda(current_step):
            if current_step < num_warmup_steps:
                factor = float(current_step) / float(max(1, num_warmup_steps))
                return max(min_factor, factor)
            progress = float(current_step - num_warmup_steps)
            progress /= float(max(1, num_training_steps - num_warmup_steps))
            if progress >= 1.0:
                return 0.0
            factor = (float(num_cycles) * progress) % 1.0
            cos = 0.5 * (1.0 + math.cos(math.pi * factor))
            lin = 1.0 - (progress * linear_decay)
            return max(min_factor, cos * lin)

        super(CosineSchedulerWithRestarts, self).__init__(optimizer, lr_lambda,
                                                          last_epoch)


### Trainer_Discriminator

In [9]:
class Trainer():
    def __init__(self,Generator,  Discriminator,  dataloader , Gen_lr, Dis_lr, alpha ,p_hint ,device , lr_scheduler=None , gamma = 0.2):
        '''
        Generator: Generator model
        Discriminator: Discriminator model
        dataloader: DataLoader object
        Gen_lr: Learning rate for the generator
        Dis_lr: Learning rate for the discriminator
        x_dim: Dimension of the input data
        device: 'cuda' or 'cpu'
        alpha: Hyperparameter for the generator loss
        p_hint: Probability of hint vector
        lr_scheduler: Learning rate scheduler
        '''
        super(Trainer, self).__init__()
        self.Generator = Generator
        self.Discriminator = Discriminator
        self.dataloader = dataloader
        self.device = device
        self.Gen_lr = Gen_lr
        self.Dis_lr = Dis_lr
        self.alpha = alpha
        self.p_hint = p_hint
        
        self.optimizer_G = optim.Adam(Generator.parameters(), lr=Gen_lr, betas=(0, 0.9))
        self.optimizer_D = optim.Adam(Discriminator.parameters(), lr=Dis_lr, betas=(0, 0.9))
        
        if lr_scheduler is not None:
            self.lr_scheduler_G = optim.lr_scheduler.ReduceLROnPlateau(self.optimizer_G, mode='min',threshold=0.00005, factor=gamma, patience=10, verbose=True)
            self.lr_scheduler_D = optim.lr_scheduler.ReduceLROnPlateau(self.optimizer_D, mode='min',threshold=0.00005, factor=gamma, patience=10, verbose=True)
        
        
        self.graph = {'iter':[],'G_loss': [], 'D_loss': [], 'MSE_train_loss': [], 'MSE_test_loss': []}
        self.total_iter = 0
        
        # self.noise_z = self.sample_noise(x_dim)
        
    def sample_noise(self, dim):
        return torch.rand(dim).to(self.device)
    
    # hint Vector Generation
    def sample_hint(self, dim, p):
        A=torch.rand(dim,dtype=torch.float16).to(self.device)
        A[A>p]=1.0
        A[A<=p]=0
        return A
    
    def discriminator_loss(self,mask,x_mask_noise,Hint):
        '''
        mask: Masked data 1:raw, 0:gen
        x_raw: Real data
        Hint: Hint vector
        '''
        x_input = torch.cat([x_mask_noise,mask], dim=1).float()
        x_gen = self.Generator(x_input)
        
        x_Hat = mask * x_mask_noise + (1 - mask) * x_gen
        
        D_prob = self.Discriminator(torch.cat([x_Hat,Hint], dim=1))
        
        return -torch.mean(mask * torch.log(D_prob + 1e-8) + (1 - mask) * torch.log(1.0 - D_prob + 1e-8))
    
    def generator_loss(self,x_raw,mask,x_mask_noise,Hint):
        '''
        x_raw: Real data
        x_gen: Generated data
        mask: Masked data 1:raw, 0:gen
        x_Hat: Combined data : mask * x_raw + (1 - mask) * x_gen
        D_prob: Discriminator output
        '''
        x_input = torch.cat([x_mask_noise,mask], dim=1)
        x_gen = self.Generator(x_input)
        x_Hat = mask * x_mask_noise + (1 - mask) * x_gen
        D_prob = self.Discriminator(torch.cat([x_Hat,Hint], dim=1))
        
        
        G_loss1 = -torch.mean((1 - mask) * torch.log(D_prob + 1e-8))
        MSE_train_loss = torch.mean((mask * x_mask_noise - mask * x_gen)**2)/torch.mean(mask)
        G_loss = G_loss1 + self.alpha * MSE_train_loss
        MSE_test_loss = torch.mean(((1 - mask) * x_raw - (1 - mask) * x_gen)**2)/torch.mean(1 - mask)
        
        return G_loss, MSE_train_loss, MSE_test_loss
    
    def test(self , test_dataloader ):
        self.Generator.eval()
        self.Discriminator.eval()
        result = {'x_raw':[],'mask': [], 'G_loss': [], 'MSE_train_loss': [], 'MSE_test_loss': []}
        
        with torch.no_grad():
            for x_raw, mask in test_dataloader:
                x_raw = x_raw.float().to(self.device).unsqueeze(-1)
                mask = mask.float().to(self.device).unsqueeze(-1) 
                x_mask_noise = mask * x_raw + (1 - mask) * self.sample_noise(x_raw.shape).to(self.device)

                G_loss, MSE_train_loss, MSE_test_loss = self.generator_loss(x_raw, mask, x_mask_noise, mask)
                result['x_raw'].append(x_raw)
                result['mask'].append(mask)
                result['G_loss'].append(G_loss.item())
                result['MSE_train_loss'].append(MSE_train_loss.item())
                result['MSE_test_loss'].append(MSE_test_loss.item())

                # print('Generator Loss: {:.4f}, MSE Train Loss: {:.4f}, MSE Test Loss: {:.4f}'.format(G_loss.item(), MSE_train_loss.item(), MSE_test_loss.item()))
        
        return result
        
    def train(self, epochs,save_path=None , test_dataloader=None):
        self.Generator.train()
        self.Discriminator.train()
        
        # 如果存在test_dataloader,则保存测试结果
        if test_dataloader is not None:
            test_graph = {'test_epochs':[],'test_G_loss': [], 'test_MSE_train_loss': [], 'test_MSE_test_loss': [] ,'test_lr':[]}

        print('Training...')
        for it in tqdm(range(epochs)):
            for x_raw, mask in self.dataloader:
                self.total_iter = self.total_iter + 1
                x_raw = x_raw.float().to(self.device).unsqueeze(1)
                mask = mask.float().to(self.device).unsqueeze(1)
                
                # print(x_raw.shape, mask.shape)
                x_mask_noise = mask * x_raw + (1 - mask) * self.sample_noise(x_raw.shape).to(self.device)
                hint = self.sample_hint(mask.shape, 1.0-self.p_hint)
                mask_hint = mask * hint
                
                # Train Discriminator
                self.optimizer_D.zero_grad()
                D_loss = self.discriminator_loss(mask,x_mask_noise,mask_hint)
                D_loss.backward()
                self.optimizer_D.step()
                
                # Train Generator
                self.optimizer_G.zero_grad()
                G_loss, MSE_train_loss, MSE_test_loss = self.generator_loss(x_raw, mask, x_mask_noise, mask_hint)
                G_loss.backward()
                self.optimizer_G.step()
                
                if self.total_iter % 256 ==0:
                    self.graph['iter'].append(self.total_iter)
                    self.graph['G_loss'].append(G_loss.item())
                    self.graph['D_loss'].append(D_loss.item())
                    self.graph['MSE_train_loss'].append(MSE_train_loss.item())
                    self.graph['MSE_test_loss'].append(MSE_test_loss.item())
                
                # print('MSE_train_loss:',MSE_train_loss.item())
                
                # break
            if test_dataloader is not None:
                result = self.test(test_dataloader)
                self.lr_scheduler_D.step(np.mean(result['MSE_test_loss']))
                self.lr_scheduler_G.step(np.mean(result['MSE_test_loss']))
                
                test_graph['test_epochs'].append(epochs)
                test_graph['test_G_loss'].append(np.mean(result['G_loss']))
                test_graph['test_MSE_train_loss'].append(np.mean(result['MSE_train_loss']))
                test_graph['test_MSE_test_loss'].append(np.mean(result['MSE_test_loss']))
                test_graph['test_lr'].append(self.optimizer_G.param_groups[0]['lr'])
                
            
            if it%2==0:
                print('Epoch: {}, Generator Loss: {:.4f}, Discriminator Loss: {:.4f}, MSE Train Loss: {:.4f}, \
                        MSE Test Loss: {:.4f}, lr :{}'.format(it, G_loss.item(), D_loss.item(), MSE_train_loss.item(), MSE_test_loss.item(), self.optimizer_G.param_groups[0]['lr']) )
                
            if save_path is not None and (it+1)%20==0:
                if not os.path.exists(save_path):
                    os.makedirs(save_path)
                self.save_checkpoint(os.path.join(save_path,'checkpoint_{}.pth'.format(it)))
        
        if save_path is not None:
            if not os.path.exists(save_path):
                    os.makedirs(save_path)
            self.save_graph(os.path.join(save_path,'Train_record.pkl'))
        
        print('Training finished!')
        
        if test_dataloader is not None:
            return test_graph
    
    def save_checkpoint(self,path):
        torch.save({'Generator_state_dict': self.Generator.state_dict(),
                    'Discriminator_state_dict': self.Discriminator.state_dict(),
                    'optimizer_G_state_dict': self.optimizer_G.state_dict(),
                    'optimizer_D_state_dict': self.optimizer_D.state_dict()}, path)
        
    def save_graph(self,path):
        with open(path, 'wb') as f:
            pickle.dump(self.graph, f)
    
    def load_checkpoint(self,path):
        checkpoint = torch.load(path)
        self.Generator.load_state_dict(checkpoint['Generator_state_dict'])
        self.Discriminator.load_state_dict(checkpoint['Discriminator_state_dict'])
        self.optimizer_G.load_state_dict(checkpoint['optimizer_G_state_dict'])
        self.optimizer_D.load_state_dict(checkpoint['optimizer_D_state_dict'])
    
    def plot_graph(self,graph):
        
        plt.figure(figsize=(16, 9))
        plt.subplot(2, 2, 1)
        plt.plot(graph['iter'], graph['G_loss'])
        plt.xlabel('Iteration')
        plt.ylabel('Generator Loss')
        plt.title('Generator Loss')
        
        plt.subplot(2, 2, 2)
        plt.plot(graph['iter'], graph['D_loss'])
        plt.xlabel('Iteration')
        plt.ylabel('Discriminator Loss')
        plt.title('Discriminator Loss')
        
        plt.subplot(2, 2, 3)
        plt.plot(graph['iter'], graph['MSE_train_loss'])
        plt.xlabel('Iteration')
        plt.ylabel('MSE Train Loss')
        plt.title('MSE Train Loss')
        
        plt.subplot(2, 2, 4)
        plt.plot(graph['iter'], graph['MSE_test_loss'])
        plt.xlabel('Iteration')
        plt.ylabel('MSE Test Loss')
        plt.title('MSE Test Loss')
        
        plt.show()
    
    def plot_test_graph(self,graph):
        plt.figure(figsize=(16, 9))
        plt.subplot(2, 2, 1)
        plt.plot(graph['test_epochs'], graph['test_G_loss'])
        plt.xlabel('Epochs')
        plt.ylabel('TestSet Generator Loss')
        plt.title('TestSet Generator Loss')
        
        plt.subplot(2, 2, 2)
        plt.plot(graph['test_epochs'], graph['test_MSE_train_loss'])
        plt.xlabel('Epochs')
        plt.ylabel('TestSet MSE Train Loss')
        plt.title('TestSet MSE Train Loss')
        
        plt.subplot(2, 2, 3)
        plt.plot(graph['test_epochs'], graph['test_MSE_test_loss'])
        plt.xlabel('Epochs')
        plt.ylabel('TestSet MSE Test Loss')
        plt.title('TestSet MSE Test Loss')
        
        plt.subplot(2, 2, 4)
        plt.plot(graph['test_epochs'], graph['test_lr'])
        plt.xlabel('Epochs')
        plt.ylabel('TestSet Learning Rate')
        plt.title('TestSet Learning Rate')
        
        plt.show()
    

### Trainer_noGAN

In [10]:
### Trainer_noGAN

class Trainer_NOGAN():
    def __init__(self,Generator,    dataloader , Gen_lr ,p_hint ,device , lr_scheduler=None , gamma = 0.2 ,No_impute_wight=1.0):
        '''
        Generator: Generator model
        Discriminator: Discriminator model
        dataloader: DataLoader object
        Gen_lr: Learning rate for the generator
        Dis_lr: Learning rate for the discriminator
        x_dim: Dimension of the input data
        device: 'cuda' or 'cpu'
        alpha: Hyperparameter for the generator loss
        p_hint: Probability of hint vector
        lr_scheduler: Learning rate scheduler
        '''
        self.Generator = Generator

        self.dataloader = dataloader
        self.device = device
        self.Gen_lr = Gen_lr
        self.p_hint = p_hint
        self.No_impute_wight=No_impute_wight
        
        self.optimizer_G = optim.Adam(Generator.parameters(), lr=Gen_lr, betas=(0, 0.9))
        
        if lr_scheduler is not None:
            self.lr_scheduler_G = CosineSchedulerWithRestarts(self.optimizer_G, num_cycles=300//100,num_training_steps=300,
                                                              num_warmup_steps=12, linear_decay=0.67,min_factor=0.1)
        else :
            self.lr_scheduler_G = None
        
        self.graph = {'iter':[],'G_loss': [], 'D_loss': [], 'MSE_train_loss': [], 'MSE_test_loss': []}
        self.total_iter = 0
        
        # self.noise_z = self.sample_noise(x_dim)
        
    def sample_noise(self, dim):
        return torch.rand(dim).to(self.device)*0.1
    
    # hint Vector Generation
    def sample_hint(self, dim, p):
        A=torch.rand(dim,dtype=torch.float16).to(self.device)
        A[A>p]=1.0
        A[A<=p]=0
        return A
    
    
    def generator_loss(self,x_raw,mask,x_mask_noise,u,Hint):
        '''
        x_raw: Real data
        x_gen: Generated data
        mask: Masked data 1:raw, 0:gen
        x_Hat: Combined data : mask * x_raw + (1 - mask) * x_gen
        '''
        # x_input = torch.cat([x_mask_noise,mask], dim=1).float()
        x_gen = self.Generator(x_raw,u,mask)
        MSE_train_loss = torch.mean((mask * x_mask_noise - mask * x_gen)**2)/torch.mean(mask)
        MSE_test_loss = torch.mean(((1 - mask) * x_raw - (1 - mask) * x_gen)**2)/torch.mean(1 - mask)
        if Hint is not None:
            temp= (1-mask).bool()
            MAPE_test_loss = torch.mean(torch.abs(( x_raw[temp] -  x_gen[temp])/ (x_raw[temp]+1e-2) ))
        else :
            MAPE_test_loss = None
        return  MSE_train_loss, MSE_test_loss ,MAPE_test_loss
    
    def test(self , test_dataloader ):
        self.Generator.eval()
        result = {'x_raw':[],'mask': [], 'G_loss': [], 'MSE_train_loss': [], 'MSE_test_loss': [], 'MAPE_test_loss': []}
        
        with torch.no_grad():
            for x_raw,u, mask in test_dataloader:
                x_raw = x_raw.float().to(self.device).unsqueeze(-1)
                u = u.float().to(self.device)
                mask = mask.float().to(self.device).unsqueeze(-1) 
                x_mask_noise = mask * x_raw + (1 - mask) * self.sample_noise(x_raw.shape).to(self.device)

                MSE_train_loss, MSE_test_loss, MAPE_test_loss = self.generator_loss(x_raw, mask, x_mask_noise,u, mask)
                result['x_raw'].append(x_raw)
                result['mask'].append(mask)
                result['MSE_train_loss'].append(MSE_train_loss.item())
                result['MSE_test_loss'].append(MSE_test_loss.item())
                result['MAPE_test_loss'].append(MAPE_test_loss .item())
                # print('Generator Loss: {:.4f}, MSE Train Loss: {:.4f}, MSE Test Loss: {:.4f}'.format(G_loss.item(), MSE_train_loss.item(), MSE_test_loss.item()))
        
        return result
        
    def train(self, epochs,save_path=None , test_dataloader=None):
        self.Generator.train()
        
        if test_dataloader is not None:
            test_graph = {'test_epochs':[], 'test_MSE_train_loss': [], 'test_MSE_test_loss': [] ,'test_lr':[]}

        print('Training...')
        for it in tqdm(range(epochs)):
            for x_raw,u, mask in self.dataloader:
                self.total_iter = self.total_iter + 1
                x_raw = x_raw.float().to(self.device).unsqueeze(-1)
                u = u.float().to(self.device)
                mask = mask.float().to(self.device).unsqueeze(-1)
                
                # print(x_raw.shape, mask.shape)
                x_mask_noise = mask * x_raw + (1 - mask) * self.sample_noise(x_raw.shape).to(self.device)
                # hint = self.sample_hint(mask.shape, 1.0-self.p_hint)
                # mask_hint = mask * hint
                
                
                # Train Generator
                self.optimizer_G.zero_grad()
                MSE_train_loss, MSE_test_loss,_ = self.generator_loss(x_raw, mask, x_mask_noise, u, None)
                total_loss = MSE_train_loss + self.No_impute_wight*MSE_test_loss
                total_loss.backward()
                self.optimizer_G.step()
                
                if self.lr_scheduler_G is not None:
                    self.lr_scheduler_G.step()
                
                if self.total_iter % 256 ==0:
                    self.graph['iter'].append(self.total_iter)
                    self.graph['MSE_train_loss'].append(MSE_train_loss.item())
                    self.graph['MSE_test_loss'].append(MSE_test_loss.item())
                
                # print('MSE_train_loss:',MSE_train_loss.item())
                
                # break
            if test_dataloader is not None:
                result = self.test(test_dataloader)
                self.lr_scheduler_G.step(np.mean(result['MSE_test_loss']))
                
                test_graph['test_epochs'].append(epochs)
                test_graph['test_MSE_train_loss'].append(np.mean(result['MSE_train_loss']))
                test_graph['test_MSE_test_loss'].append(np.mean(result['MSE_test_loss']))
                test_graph['test_lr'].append(self.optimizer_G.param_groups[0]['lr'])
                
            
            if it%2==0:
                print('Epoch: {}, Generator Loss: {:.4f}, MSE Train Loss: {:.4f},  MSE Test Loss: {:.4f}, lr :{}' \
                      .format(it, total_loss.item(), MSE_train_loss.item(), MSE_test_loss.item(), self.optimizer_G.param_groups[0]['lr']) )
                
            if save_path is not None and (it+1)%20==0:
                if not os.path.exists(save_path):
                    os.makedirs(save_path)
                self.save_checkpoint(os.path.join(save_path,'checkpoint_{}.pth'.format(it)))
        
        if save_path is not None:
            if not os.path.exists(save_path):
                    os.makedirs(save_path)
            self.save_graph(os.path.join(save_path,'Train_record.pkl'))
        
        print('Training finished!')
        
        if test_dataloader is not None:
            return test_graph
    
    def save_checkpoint(self,path):
        torch.save({'Generator_state_dict': self.Generator.state_dict(),
                    'optimizer_G_state_dict': self.optimizer_G.state_dict()}, path)
        
    def save_graph(self,path):
        with open(path, 'wb') as f:
            pickle.dump(self.graph, f)
    
    def load_checkpoint(self,path):
        checkpoint = torch.load(path)
        self.Generator.load_state_dict(checkpoint['Generator_state_dict'])
        self.optimizer_G.load_state_dict(checkpoint['optimizer_G_state_dict'])

    def plot_graph(self,graph):

        
        plt.figure(figsize=(16, 9))
        # plt.subplot(2, 2, 1)
        # plt.plot(graph['iter'], graph['G_loss'])
        # plt.xlabel('Iteration')
        # plt.ylabel('Generator Loss')
        # plt.title('Generator Loss')
        
        
        plt.subplot(1, 2, 1)
        plt.plot(graph['iter'], graph['MSE_train_loss'])
        plt.xlabel('Iteration')
        plt.ylabel('MSE Train Loss')
        plt.title('MSE Train Loss')
        
        plt.subplot(1, 2, 2)
        plt.plot(graph['iter'], graph['MSE_test_loss'])
        plt.xlabel('Iteration')
        plt.ylabel('MSE Test Loss')
        plt.title('MSE Test Loss')
        
        plt.show()
    
    def plot_test_graph(self,graph):

        
        plt.figure(figsize=(16, 9))

        plt.subplot(1, 2, 1)
        plt.plot(graph['test_epochs'], graph['test_MSE_train_loss'])
        plt.xlabel('Epochs')
        plt.ylabel('TestSet MSE Train Loss')
        plt.title('TestSet MSE Train Loss')
        
        plt.subplot(1, 2, 2)
        plt.plot(graph['test_epochs'], graph['test_MSE_test_loss'])
        plt.xlabel('Epochs')
        plt.ylabel('TestSet MSE Test Loss')
        plt.title('TestSet MSE Test Loss')
        
        # plt.subplot(2, 2, 4)
        # plt.plot(graph['test_epochs'], graph['test_lr'])
        # plt.xlabel('Epochs')
        # plt.ylabel('TestSet Learning Rate')
        # plt.title('TestSet Learning Rate')
        
        plt.show()
    

### load_dataset

In [ ]:
source_train_path = r'D:\WorkPath\Models\ImputeFormer\Data\source_train_SanDiego'
source_train_files = os.listdir(source_train_path)
source_train_files = [os.path.join(source_train_path, file) for file in source_train_files]
source_train_data = []

for file_path in source_train_files:
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
    source_train_data.append(data)

data = ConcatDataset(source_train_data)
data_loader = DataLoader(data, batch_size=32, shuffle=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

len(data)

In [12]:
# train_size = int(len(data) * 0.8)
# test_size = len(data) - train_size
# train_dataset, test_dataset = torch.utils.data.random_split(data, [train_size, test_size])
# data_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# print(len(train_dataset))
# print(len(test_dataset))

### Training_NoGAN

In [13]:
device= torch.device("cuda" if torch.cuda.is_available() else "cpu") 
model = ImputeFormer(num_nodes=32,input_dim=3,output_dim=1,input_embedding_dim=32,
                     learnable_embedding_dim=96, feed_forward_dim=256,num_temporal_heads=4,
                     num_layers= 3,dim_proj= 8,windows=48).to(device)

lr=0.0001
trainer = Trainer_NOGAN(model,data_loader,Gen_lr=lr,p_hint=0,device=device,lr_scheduler=None,No_impute_wight=1)
# trainer.load_checkpoint(r'D:\WorkPath\Models\ImputeFormer\Checkpoints\checkpoint_119.pth')
# trainer.train(epochs=40, save_path=r'D:\WorkPath\Models\ImputeFormer\Checkpoints') 

In [ ]:
project_path = r'D:\WorkPath\Models\ImputeFormer'
test_path = os.path.join(project_path , r'Data\source_test_PEMS04') 
test_files = os.listdir(test_path)
test_files = [os.path.join(test_path, file) for file in test_files]
test_record = {'data_name':[],'MSE_train_loss':[] , 'MSE_test_loss':[], 'MAPE_test_loss':[]}

trainer.load_checkpoint(r'D:\WorkPath\Models\ImputeFormer\SAVE\checkpoint_20240918.pth')

for file_path in test_files:
    with open(file_path, 'rb') as f:
        test_data = pickle.load(f)
    test_data_loader = DataLoader(test_data, batch_size=16, shuffle=False)
    result=trainer.test(test_data_loader)
    test_record['data_name'].append(file_path)
    test_record['MSE_train_loss'].append(np.mean(result['MSE_train_loss'][1:]))
    test_record['MSE_test_loss'].append(np.mean(result['MSE_test_loss'][1:]))
    test_record['MAPE_test_loss'].append(np.mean(result['MAPE_test_loss'][1:]))
test_record = pd.DataFrame(test_record)
test_record['route']=test_record['data_name'].apply(lambda x :x.split('_')[5])
test_record['start']=test_record['data_name'].apply(lambda x :x.split('_')[-3])
test_record['miss_rate']=test_record['data_name'].apply(lambda x :x.split('_')[-2])
test_record['type']=test_record['data_name'].apply(lambda x :x.split('_')[-1][:-4])
test_record=test_record[['route','start','miss_rate','type','MSE_train_loss','MSE_test_loss','MAPE_test_loss']]
test_record=test_record.sort_values(['route','type','start'])
test_record

In [14]:
trainer.save_checkpoint(r'D:\WorkPath\Models\ImputeFormer\SAVE\checkpoint_20241007_1.pth')

In [ ]:
test_record.groupby('type')[['MSE_test_loss','MAPE_test_loss']].mean()

In [ ]:
device= torch.device("cuda" if torch.cuda.is_available() else "cpu") 
model = ImputeFormer(num_nodes=32,input_dim=3,output_dim=1,input_embedding_dim=32,
                     learnable_embedding_dim=96, feed_forward_dim=256,num_temporal_heads=4,
                     num_layers= 3,dim_proj= 8,windows=48).to(device)

lr=0.0001
trainer = Trainer_NOGAN(model,data_loader,Gen_lr=lr,p_hint=0,device=device,lr_scheduler=None,No_impute_wight=1)

project_path = r'D:\WorkPath\Models\ImputeFormer'
test_path = os.path.join(project_path , r'Data\target_test') 
test_files = os.listdir(test_path)
test_files = [os.path.join(test_path, file) for file in test_files]
test_record = {'data_name':[],'MSE_train_loss':[] , 'MSE_test_loss':[], 'MAPE_test_loss':[]}

trainer.load_checkpoint(r'D:\WorkPath\Models\ImputeFormer\Checkpoints\checkpoint_199.pth')

for file_path in test_files:
    with open(file_path, 'rb') as f:
        test_data = pickle.load(f)
    test_data_loader = DataLoader(test_data, batch_size=16, shuffle=False)
    result=trainer.test(test_data_loader)
    test_record['data_name'].append(file_path)
    test_record['MSE_train_loss'].append(np.mean(result['MSE_train_loss'][1:]))
    test_record['MSE_test_loss'].append(np.mean(result['MSE_test_loss'][1:]))
    test_record['MAPE_test_loss'].append(np.mean(result['MAPE_test_loss'][1:]))
test_record = pd.DataFrame(test_record)
test_record['route']=test_record['data_name'].apply(lambda x :x.split('_')[4])
test_record['start']=test_record['data_name'].apply(lambda x :x.split('_')[-3])
test_record['miss_rate']=test_record['data_name'].apply(lambda x :x.split('_')[-2])
test_record['type']=test_record['data_name'].apply(lambda x :x.split('_')[-1][:-4])
test_record=test_record[['route','start','miss_rate','type','MSE_train_loss','MSE_test_loss','MAPE_test_loss']]
test_record=test_record.sort_values(['route','type','start'])
test_record

In [ ]:
test_record['MSE_test_loss'].mean()

In [ ]:
test_record.groupby('type')[['MSE_test_loss','MAPE_test_loss']].mean()